# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [53]:
! git clone https://github.com/cutiepie1011/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

fatal: destination path 'exam_2025' already exists and is not an empty directory.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [54]:
mySeed = 350

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [25]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

In [26]:
# Pour estimer les coefficients theta_k on peut utiliser la méthodes de la régression linéraire multiple
# (moindres carrés ordinaires), qui consiste à trouver les coefficients qui minimisent la somme des carrés
# des différences entre les valeurs prédites et les valeurs réelles.


In [27]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Extraire les prédicteurs (X) et les cibles (t)
x  = train_set['inputs']
t = train_set['targets']

# Création du modèle d'entrainement : régression linéaire
modele = LinearRegression()

# Ajuster le modèle aux données d'entraînement
modele.fit(x, t)

print("Coefficients :", modele.coef_)
print("Interception :", modele.intercept_)

Coefficients : [3.44301622 3.59207667 7.17464915]
Interception : 17.43049692114625


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

In [28]:
#L'architecture = c'est un réseau de neurone constitué d'une seule couche linéaire ce qui suifit car le
# problème est linéaire. Cette couche calcule une combinaison linéaire pondérée des entrées en y ajoutant un
# biais. Le réseau neuronal sera entrainé en utilisant la descente du gradient stochastique.

# Expressivité  : t = theta0 + theta1.x + theta2.y + theta3.z
# En terme de performance en généralisation : un modèle plus simple avec moins de paramètres est moins probable
# d'ajouter des données d'entrainement ==> minimiser le surapprentissage.
# En terme de performance : Ce qui conduit automatiquement a un entrainement plus rapide et plus efficace et
# et une mailleure généralisation sur des données invisibles.

In [29]:
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# A coder :
import torch.nn as nn
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [30]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:

       # Propagation avant
        outputs = mySimpleNet(batch_inputs.float())
        loss = criterion(outputs, batch_targets.float().unsqueeze(1))

        # Rétropropagation et optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Le valeur de la perte diminue bien à chaque époque, puis stagne à une valeur faible , ce qui est bon signe
# sur l'aprentissage et que le modèle a bien appris à modéliser les données d'entraînement

Epoch [1/500], Loss: 285.1299
Epoch [2/500], Loss: 168.1904
Epoch [3/500], Loss: 109.7982
Epoch [4/500], Loss: 61.0179
Epoch [5/500], Loss: 41.4430
Epoch [6/500], Loss: 30.5691
Epoch [7/500], Loss: 16.5878
Epoch [8/500], Loss: 15.6148
Epoch [9/500], Loss: 10.5494
Epoch [10/500], Loss: 5.9747
Epoch [11/500], Loss: 5.3427
Epoch [12/500], Loss: 5.1720
Epoch [13/500], Loss: 3.5665
Epoch [14/500], Loss: 4.5485
Epoch [15/500], Loss: 4.4740
Epoch [16/500], Loss: 5.1485
Epoch [17/500], Loss: 4.8590
Epoch [18/500], Loss: 4.5817
Epoch [19/500], Loss: 5.0041
Epoch [20/500], Loss: 5.6978
Epoch [21/500], Loss: 4.2130
Epoch [22/500], Loss: 4.8267
Epoch [23/500], Loss: 4.0846
Epoch [24/500], Loss: 4.0418
Epoch [25/500], Loss: 4.0144
Epoch [26/500], Loss: 4.0567
Epoch [27/500], Loss: 4.1564
Epoch [28/500], Loss: 4.2055
Epoch [29/500], Loss: 3.6420
Epoch [30/500], Loss: 3.3631
Epoch [31/500], Loss: 4.2401
Epoch [32/500], Loss: 3.8739
Epoch [33/500], Loss: 3.8838
Epoch [34/500], Loss: 4.0896
Epoch [35/5

**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [31]:
# Les estimation des thetak sont stockés dans les paramètres de la couche linéaire du modèle
 #(de l'objet mySimpleNet). On peut accéder au poids via .linear.weight et au biais via .linear.bias.

In [32]:
# Extraire les trois poids
weights = mySimpleNet.linear.weight.data.numpy()

# Extraire le biais (θ0)
bias = mySimpleNet.linear.bias.data.numpy()

print("Poids (θ1, θ2, θ3) :", weights)
print("Biais (θ0) :", bias)

Poids (θ1, θ2, θ3) : [[3.4428918 3.5909214 7.174001 ]]
Biais (θ0) : [17.429174]


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [37]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import torch

# Préparer les données de test
X_test = test_set['inputs']
y_test = test_set['targets']
# Prédictions avec le réseau de neurones
X_test_tensor = torch.tensor(X_test).float()
predictions_nn = mySimpleNet(X_test_tensor).detach().numpy()
# Prédiction avec la régression linéaire
model = LinearRegression()
model.fit(train_set['inputs'], train_set['targets'])
predictions_lr = model.predict(X_test)
# Comparaison des performances
mse_nn = mean_squared_error(y_test, predictions_nn)
mse_lr = mean_squared_error(y_test, predictions_lr)

print("MSE du réseau de neurones:", mse_nn)
print("MSE de la régression linéaire:", mse_lr)

#Les moyennes sont très similaires ce qui montre que les deux
# approches donnent des performances identiques en termes de généralisation.


MSE du réseau de neurones: 4.055477822302178
MSE de la régression linéaire: 4.055641510168173


\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [39]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [ ]:
# Il s'agit FCN , fully convolutional Network
#, plus précisément un réseau de neurones adapté pour le traitement des séries temporelles.

In [ ]:
# Nb de paramètres dans self.Down1: (calcul "à la main")
# La formule = entrée * taille_noyau * sortie + sortie
# ici : entrée = 64 canaux , sortie = 2*64 = 128 canaux et taille_noyau = 3
# nb_para = (64*3*128) + 128 = 24 704.


# Nb de paramètres au total: 2872641 voir calcul si dessous

In [41]:
import torch

model = causalFCN()

total_params = sum(p.numel() for p in model.parameters())

print(f"Nombre total de paramètres : {total_params}")

Nombre total de paramètres : 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

In [ ]:
# Les mécanismes pour réduire la taille du vecteur sont :
# Down_causal : des couches de convolution causales ( extraient des caractéristiques temporelles tout en respectant la causalité)
# et de pooling(stride) en  diminuant la longueur de la séquence, dans les couches descendantes.
# Pour la restitution via Upsampling : les couches Up_causal utilisent des méthodes de upsampling.
# Chaque couche Up_causal utilise une convolution avec un noyau et un stride,
# et fusionne les informations des couches correspondantes de descente pour restaurer progressivement
# la résolution du vecteur d'entrée.


**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

In [ ]:
# Le champ réceptif en sortie de self.inc est de 5 éléments.
# Cela résulte de deux convolutions successives avec un noyau de taille 3 et une dilatation de 1,
# chacune ajoutant 3 éléments au champ réceptif, donnant ainsi un total de 1 + (3 -1)* 1 * 2 = 5.


**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [43]:
import torch

model = causalFCN()

# Créer deux entrées
input1 = torch.zeros(1, 1, 10000)
input2 = torch.zeros(1, 1, 10000)
# modifier la composante à l'indice 5000
input2[0, 0, 5000] = 1

# Calculer les sorties
output1 = model(input1)
output2 = model(input2)

indices_diff = torch.nonzero(output1 - output2)

# Le champ réceptif est la plage d'indices où les sorties diffèrent
receptive_field_size = indices_diff[-1, -1] - indices_diff[0, -1] + 1

print("La taille du champ réceptif pour y_5000 est :", receptive_field_size.item())

La taille du champ réceptif pour y_5000 est : 10000


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



In [50]:
import torch


# Instancier le modèle
model = causalFCN()

input1 = torch.zeros(1, 1, 10000)
input2 = torch.zeros(1, 1, 10000)
input2[0, 0, 5001:] = torch.rand(4999)

# Calculer les sorties correspondantes
output1 = model(input1)
output2 = model(input2)

# Comparer la valeur de y_5000 dans les deux sorties
print("y_5000_in1:", output1[0, 0, 5000].item())
print("y_5000_in2 :", output2[0, 0, 5000].item())

y_5000_in1: 0.027746466919779778
y_5000_in2 : 0.25076577067375183


\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?
" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

In [ ]:
# positive samples =
# negative samples =

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?